In [1]:
import os
import sys

from dotenv import load_dotenv

current_dir = os.getcwd()
while True:
    if os.path.basename(current_dir) == "pybye":
        break
    parent_dir = os.path.dirname(current_dir)
    if parent_dir == current_dir:
        raise Exception("pybye not found")
    current_dir = parent_dir

env_path = os.path.join(current_dir, ".env")
load_dotenv(dotenv_path=env_path)

sys.path.append(current_dir)

In [37]:
import pandas as pd
from config import MEXC_DATA_PATH

kline = pd.read_csv(f"C:/Users/Slava/crypto/pybye/{MEXC_DATA_PATH}kline/mexc_kline_WIF_USDT_futures_Min15.csv")

In [ ]:
kline.head(3)

In [ ]:
kline["Time"] = pd.to_datetime(kline["Time"], utc=True)
kline.Time.head(3)

In [ ]:
kline.set_index("Time", inplace=True)
kline = kline[["Open", "High", "Low", "Close", "Volume"]]
kline.head(3)

In [ ]:
import mplfinance as mpf

mpf.plot(kline.head(100), type='candle', style='charles', title='Candlestick Chart', volume=True, figratio=(35, 9))

In [17]:
# distance. Specifies the minimum number of data points between consecutive peaks.
# This helps avoid detecting too many peaks too close to each other.

# prominence. A peak’s prominence represents how much it stands out from the surrounding data.
# The higher the prominence, the more significant the peak.

In [ ]:
import scipy as sp


# Define the distance between strong peaks (in days).
strong_peak_distance = 10

# Define the prominence (how high the peaks are compared to their surroundings).
strong_peak_prominence = 0.5

# Find the strong peaks in the 'high' price data
strong_peaks, _ = sp.signal.find_peaks(
  kline['High'],
  distance=strong_peak_distance,
  prominence=strong_peak_prominence
)

# Extract the corresponding high values of the strong peaks
strong_peaks_values = kline.iloc[strong_peaks]["High"].values.tolist()
strong_peaks_values[:4]

In [ ]:
import numpy as np

# Create a list of horizontal lines to plot as resistance levels
add_plot = [mpf.make_addplot(np.full(kline.shape[0], resistance), color='r', linestyle='--') for resistance in strong_peaks_values]

# Plot the candlestick chart with resistance lines
mpf.plot(
    kline, 
    type='candle', 
    style='charles', 
    title='Candlestick Chart with Strong Resistance Lines',
    volume=True, 
    addplot=add_plot,
    figratio=(35, 9)
)

In [33]:
# Define the shorter distance between general peaks (in days)
# This controls how far apart peaks need to be to be considered separate.
peak_distance = 5

# Define the width (vertical distance) where peaks within this range will be grouped together.
# If the high prices of two peaks are closer than this value, they will be merged into a single resistance level.
peak_rank_width = 1

# Define the threshold for how many times the stock has to reject a level
# Before it becomes a resistance level
resistance_min_pivot_rank = 2

# Find general peaks in the stock's 'high' prices based on the defined distance between them.
# The peaks variable will store the indices of the high points in the 'high' price data.
peaks, _ = sp.signal.find_peaks(kline['High'], distance=peak_distance)

# Initialize a dictionary to track the rank of each peak
peak_to_rank = {peak: 0 for peak in peaks}

# Loop through all general peaks to compare their proximity and rank them
for i, current_peak in enumerate(peaks):
    # Get the current peak's high price
    current_high = kline.iloc[current_peak]["High"]
    
    # Compare the current peak with previous peaks to calculate rank based on proximity
    for previous_peak in peaks[:i]:
        if abs(current_high - kline.iloc[previous_peak]["High"]) <= peak_rank_width:
            # Increase rank if the current peak is close to a previous peak
            peak_to_rank[current_peak] += 1

In [ ]:
# Initialize the list of resistance levels with the strong peaks already identified.
resistances = strong_peaks_values

# Now, go through each general peak and add it to the resistance list if its rank meets the minimum threshold.
for peak, rank in peak_to_rank.items():
    # If the peak's rank is greater than or equal to the resistance_min_pivot_rank, 
    # it means this peak level has been rejected enough times to be considered a resistance level.
    if rank >= resistance_min_pivot_rank:
        # Append the peak's high price to the resistances list, adding a small offset (1e-3) 
        # to avoid floating-point precision issues during the comparison.
        resistances.append(kline.iloc[peak]["High"] + 1e-3)

# Sort the list of resistance levels so that they are in ascending order.
resistances.sort()
resistances[:4]

In [ ]:
# Initialize a list to hold bins of resistance levels that are close to each other.
resistance_bins = []

# Start the first bin with the first resistance level.
current_bin = [resistances[0]]

# Loop through the sorted resistance levels.
for r in resistances:
    # If the difference between the current resistance level and the last one in the current bin 
    # is smaller than a certain threshold (defined by peak_rank_w_pct), add it to the current bin.
    if r - current_bin[-1] < peak_rank_width:
        current_bin.append(r)
    else:
        # If the current resistance level is far enough from the last one, close the current bin
        # and start a new one.
        resistance_bins.append(current_bin)
        current_bin = [r]

# Append the last bin.
resistance_bins.append(current_bin)

# For each bin, calculate the average of the resistances within that bin.
# This will produce a clean list of resistance levels where nearby peaks have been merged.
resistances = [np.mean(bin) for bin in resistance_bins]
resistances[:4]

In [ ]:
# Create a list of horizontal lines to plot as resistance levels
add_plot = [mpf.make_addplot(np.full(kline.shape[0], resistance), color='r', linestyle='--') for resistance in resistances]

# Plot the candlestick chart with resistance lines
mpf.plot(
    kline, 
    type='candle', 
    style='charles', 
    title='Candlestick Chart with Strong Resistance Lines',
    volume=True, 
    addplot=add_plot,
    figratio=(35, 9)
)